<a href="https://colab.research.google.com/github/asheemita97-pixel/Spatial-Localization-Dsz/blob/main/E1dispersed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# ---------------- PARAMETERS ----------------
diameter_um = 1.0
R = diameter_um / 2.0          # membrane radius
margin_um = 0.0025              # exclusion distance from membrane
target_points = 10000          # desired number of points
# --------------------------------------------

usable_R = R - margin_um

# ---------------------------------------------------------
# HEXAGONAL LATTICE (most uniform 2D distribution)
# ---------------------------------------------------------
# Area available
area = np.pi * usable_R**2

# spacing required for ~target_points
# hex packing density = sqrt(3)/2 * a^2 per point
a = np.sqrt((2 * area) / (np.sqrt(3) * target_points))

dx = a
dy = np.sqrt(3)/2 * a

# create lattice large enough to cover circle
x_vals = np.arange(-usable_R, usable_R + dx, dx)
y_vals = np.arange(-usable_R, usable_R + dy, dy)

points = []

for j, y in enumerate(y_vals):
    shift = 0 if j % 2 == 0 else dx/2   # hex offset
    for x in x_vals:
        xx = x + shift
        if xx**2 + y**2 <= usable_R**2:
            points.append((xx, y))

points = np.array(points)
x = points[:,0]
y = points[:,1]
r = np.sqrt(x**2 + y**2)

print(f"Generated points: {len(points)}")
print(f"Usable radius: {usable_R:.4f} µm")

# ---------------------------------------------------------
# SAVE CSV
# ---------------------------------------------------------
df = pd.DataFrame({
    "x_um": x,
    "y_um": y,
    "r_um": r,
    "distance_from_membrane_um": R - r
})
csv_name = "uniform_hex_points_cell.csv"
df.to_csv(csv_name, index=False)
print(f"Saved: {csv_name}")

# ---------------------------------------------------------
# VISUALIZATION
# ---------------------------------------------------------
fig, ax = plt.subplots(figsize=(6,6))
ax.scatter(x, y, s=3)

# membrane
membrane = plt.Circle((0,0), R, fill=False, lw=2)
ax.add_artist(membrane)

# allowed boundary
inner = plt.Circle((0,0), usable_R, fill=False, linestyle='--', lw=2)
ax.add_artist(inner)

ax.set_aspect('equal')
ax.set_xlim(-R*1.1, R*1.1)
ax.set_ylim(-R*1.1, R*1.1)
ax.set_xlabel("x (µm)")
ax.set_ylabel("y (µm)")
ax.set_title("Uniform hexagonal distribution inside cell")

plt.show()


In [ ]:
# Colab-ready cells. Full updated code.
# Changes:
# - E1 is now randomly dispersed across the usable domain (both the hex-lattice CSV and the PDE grid use a random initial E1)
# - E1 diffuses over time with D_E1 = 0.01 µm^2/s (explicit FTCS update; dt/dx satisfy stability condition)
# - All visualizations, CSV / npy outputs, and the flow are kept as in your original script
# - A reproducible random seed option is provided

# ---------------- Cell 1: hex lattice + random E1 dispersion (area-averaged = desired_E1_uM) ----------------
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# --------------- User parameters (from your code) ----------------
diameter_um = 1.0
R = diameter_um / 2.0           # membrane radius (µm)
margin_um = 0.0025              # exclusion distance from membrane (µm)
target_points = 10000           # desired number of lattice points (used to pick spacing)
# -----------------------------------------------------------------

usable_R = R - margin_um
if usable_R <= 0:
    raise ValueError("usable radius <= 0. Choose smaller margin or larger diameter.")

# shell parameters (no longer used to place E1; kept for plotting context)
shell_width = 0.15              # width from the outer membrane inward (µm)
shell_inner_radius = max(0.0, usable_R - shell_width)
shell_outer_radius = usable_R   # points exist only up to usable_R (we don't place points inside the exclusion margin)

# desired effective (area-averaged) concentration inside the entire cell (for E1)
desired_conc_uM = 60.9          # µM

# reproducible randomness
random_seed = 42
np.random.seed(random_seed)

# -------------------- build hex lattice (your original method) --------------------
area_usable = np.pi * usable_R**2
a = np.sqrt((2 * area_usable) / (np.sqrt(3) * target_points))  # lattice spacing (approx)
dx = a
dy = np.sqrt(3)/2 * a

x_vals = np.arange(-usable_R, usable_R + dx, dx)
y_vals = np.arange(-usable_R, usable_R + dy, dy)

points = []
for j, y in enumerate(y_vals):
    shift = 0.0 if j % 2 == 0 else dx/2.0
    for x in x_vals:
        xx = x + shift
        if xx**2 + y**2 <= usable_R**2:
            points.append((xx, y))

points = np.array(points)
x = points[:,0]
y = points[:,1]
r = np.sqrt(x**2 + y**2)

n_points = len(points)
usable_area = np.pi * usable_R**2

print(f"Generated lattice points: {n_points}")
print(f"Usable radius = {usable_R:.6f} µm, shell inner radius = {shell_inner_radius:.6f} µm")
print(f"Usable area (points cover) = {usable_area:.6e} µm^2")
print()

# -------------------- create RANDOM dispersed E1 over lattice points --------------------
# create positive random field then scale so area-averaged conc (over the whole cell) = desired_conc_uM
rand_field = np.random.rand(n_points)  # uniform [0,1)
mean_rand = rand_field.mean()
if mean_rand <= 0:
    raise RuntimeError("random field mean is zero; unexpected")
# scale to desired mean (units: µM per lattice point)
conc_uM = rand_field * (desired_conc_uM / mean_rand)

# bookkeeping
area_per_point = usable_area / n_points
effective_conc_from_points = (conc_uM.sum() * area_per_point) / (np.pi * R**2)

# -------------------- save CSV --------------------
in_shell_mask = (r >= shell_inner_radius) & (r <= shell_outer_radius)
df = pd.DataFrame({
    "x_um": x,
    "y_um": y,
    "r_um": r,
    "distance_from_membrane_um": R - r,
    "in_shell": in_shell_mask.astype(int),
    "E1_conc_uM": conc_uM
})
csv_name = "hex_points_with_random_E1_concentration.csv"
df.to_csv(csv_name, index=False)

# -------------------- print diagnostics --------------------
print(f"Area per lattice point (approx) = {area_per_point:.6e} µm^2")
print(f"Discrete effective concentration computed from points = {effective_conc_from_points:.6f} µM")
print(f"Target desired concentration = {desired_conc_uM:.6f} µM")
print(f"CSV saved to: {csv_name}")
print()

# -------------------- quick visualization --------------------
fig, ax = plt.subplots(1,2, figsize=(12,6))
# scatter coloring by conc
sc = ax[0].scatter(x, y, c=conc_uM, s=6, cmap='viridis', vmin=0, vmax=conc_uM.max())
ax[0].add_artist(plt.Circle((0,0), R, fill=False, lw=1.5, edgecolor='black'))         # membrane
ax[0].add_artist(plt.Circle((0,0), usable_R, fill=False, lw=1.5, linestyle='--', edgecolor='red'))  # usable outer boundary
ax[0].add_artist(plt.Circle((0,0), shell_inner_radius, fill=False, lw=1.0, linestyle=':', edgecolor='orange'))  # shell inner bound
ax[0].set_aspect('equal'); ax[0].set_title("Initial RANDOM E1 concentration (µM) per lattice point")
ax[0].set_xlim(-R*1.05, R*1.05); ax[0].set_ylim(-R*1.05, R*1.05)
plt.colorbar(sc, ax=ax[0], label='E1 conc (µM)')

# show mask of shell points (binary) for context
ax[1].scatter(x, y, c=in_shell_mask, s=6, cmap='gray_r')
ax[1].add_artist(plt.Circle((0,0), R, fill=False, lw=1.5, edgecolor='black'))
ax[1].add_artist(plt.Circle((0,0), usable_R, fill=False, lw=1.5, linestyle='--', edgecolor='red'))
ax[1].add_artist(plt.Circle((0,0), shell_inner_radius, fill=False, lw=1.0, linestyle=':', edgecolor='orange'))
ax[1].set_aspect('equal'); ax[1].set_title("Shell mask (1=in shell)")
ax[1].set_xlim(-R*1.05, R*1.05); ax[1].set_ylim(-R*1.05, R*1.05)

plt.show()


# ---------------- Cell 2: PDE simulation with E1 random initial field and E1 diffusion ----------------
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import imageio
import os
from datetime import datetime

# ------------------ Parameters (edit as needed) ------------------
out_dir = "/content/sim_out"
os.makedirs(out_dir, exist_ok=True)

diameter_um = 1.0
R = diameter_um / 2.0            # µm
margin_um = 0.0025               # excluded margin (µm)
usable_R = R - margin_um

# enzyme shell geometry (kept for plotting context)
shell_width = 0.15               # µm inward from usable_R
shell_outer = usable_R
shell_inner = max(0.0, usable_R - shell_width)

# concentrations & kinetics
S0_annulus_mM = 0.134            # initial substrate in excluded annulus (mM)
desired_E1_uM = 60.9             # area-averaged enzyme conc (µM)
kcat = 0.93 / 60.0               # s^-1
Km_mM = 0.00111                  # mM

# diffusion constants
D_S = 0.01                       # µm^2/s for substrate S (explicit)
D_I1 = 0.01                      # µm^2/s for product I1 (implicit)
D_E1 = 0.01                      # µm^2/s for enzyme E1 (explicit as requested)

# numerical grid & time
dx = 0.01                        # µm
dt = 0.002                       # s  (explicit stability: dt <= dx^2/(4*D) ; with dx=0.01 and D=0.01 -> dt<=0.0025)
t_max = 3                      # total simulation time (s)
save_frames = True
fps = 12

# reproducible randomness
random_seed = 42
np.random.seed(random_seed)

# -----------------------------------------------------------------

# Derived grid
nx = int(np.ceil((2*R) / dx)) + 1
x = np.linspace(-R, R, nx)
y = np.linspace(-R, R, nx)
X, Y = np.meshgrid(x, y)
r_grid = np.sqrt(X**2 + Y**2)
mask = r_grid <= R + 1e-12       # domain mask (boolean)
area_per_cell = dx * dx

# Build enzyme (E1) RANDOM field (mM) with area-averaged concentration = desired_E1_uM
c_target_mM = desired_E1_uM / 1000.0
rand_field_grid = np.random.rand(*r_grid.shape) * mask.astype(float)  # zeros outside mask
mean_on_mask = rand_field_grid[mask].mean()
if mean_on_mask <= 0:
    raise RuntimeError("random field mean on mask is zero; unexpected")
E1 = rand_field_grid * (c_target_mM / mean_on_mask)  # mM units, mean over mask => c_target_mM

# Initialize fields
S = np.zeros_like(r_grid)
annulus_mask = (r_grid >= usable_R - 1e-12) & (r_grid <= R + 1e-12)
S[annulus_mask] = S0_annulus_mM
I1 = np.zeros_like(r_grid)

initial_S_total = np.sum(S[mask]) * area_per_cell

print(f"Grid: {nx}x{nx}, dx={dx} µm, dt={dt}s, steps={int(np.ceil(t_max/dt))}")
print(f"Initial E1 mean (µM) = {np.mean(E1[mask])*1000.0:.3f} µM; D_E1={D_E1}")
print(f"D_S={D_S}, D_I1={D_I1}")
print(f"Initial total S (mM·µm^2) = {initial_S_total:.6e}")

# ------------------ helper: explicit laplacian (Neumann treatment) ------------------
def laplacian_2d(Z, domain_mask, dx):
    Zc = Z
    left = Zc.copy(); right = Zc.copy(); up = Zc.copy(); down = Zc.copy()
    left[:,1:] = Zc[:,:-1]
    right[:,:-1] = Zc[:,1:]
    up[1:,:] = Zc[:-1,:]
    down[:-1,:] = Zc[1:,:]
    mask_left = np.ones_like(domain_mask, dtype=bool)
    mask_right = np.ones_like(domain_mask, dtype=bool)
    mask_up = np.ones_like(domain_mask, dtype=bool)
    mask_down = np.ones_like(domain_mask, dtype=bool)
    mask_left[:,1:] = domain_mask[:,:-1]
    mask_right[:,:-1] = domain_mask[:,1:]
    mask_up[1:,:] = domain_mask[:-1,:]
    mask_down[:-1,:] = domain_mask[1:,:]
    neigh_sum = (np.where(mask_left, left, Zc) +
                 np.where(mask_right, right, Zc) +
                 np.where(mask_up, up, Zc) +
                 np.where(mask_down, down, Zc))
    lap = (neigh_sum - 4.0 * Zc) / (dx*dx)
    lap[~domain_mask] = 0.0
    return lap

# ------------------ Build sparse Laplacian matrix for masked nodes (Neumann handling) ------------------
mask_positions = np.array(np.nonzero(mask)).T  # list of (i,j)
n_mask = mask_positions.shape[0]
idx_map = -np.ones_like(mask, dtype=int)
for k, (i, j) in enumerate(mask_positions):
    idx_map[i, j] = k

rows = []
cols = []
data = []

for k, (i, j) in enumerate(mask_positions):
    neighbors = [(i, j-1), (i, j+1), (i-1, j), (i+1, j)]
    neighbors_present = 0
    for (ni, nj) in neighbors:
        if 0 <= ni < nx and 0 <= nj < nx and mask[ni, nj]:
            neighbors_present += 1
            rows.append(k)
            cols.append(idx_map[ni, nj])
            data.append(1.0)
    rows.append(k); cols.append(k); data.append(-float(neighbors_present))

L = sp.coo_matrix((data, (rows, cols)), shape=(n_mask, n_mask)).tocsr()
Llap = L / (dx*dx)    # Laplacian operator (applies to vector of masked nodes)

# ------------------ Prepare implicit solver for I1 diffusion ------------------
I_mat = sp.eye(n_mask, format='csc')
A = I_mat - dt * D_I1 * Llap.tocsc()
lu = spla.splu(A)  # sparse LU factorization

# ------------------ helper to render figure to RGB ------------------
def fig_to_rgb_array(fig):
    fig.canvas.draw()
    buf, (w, h) = fig.canvas.print_to_buffer()
    arr = np.frombuffer(buf, dtype=np.uint8).reshape((h, w, 4))
    return arr[:, :, :3].copy()

# ------------------ Time-stepping ------------------
steps = int(np.ceil(t_max / dt))
frame_interval = max(1, steps // 80)
frames_S = []
frames_I1 = []
area_mask_count = np.sum(mask)

times = []
totS = []
totI = []

for n in range(steps):
    # E1: explicit diffusion (enzyme moves but is not consumed)
    lapE = laplacian_2d(E1, mask, dx)
    E1_new = E1 + dt * (D_E1 * lapE)
    E1_new[~mask] = 0.0
    E1_new[E1_new < 0] = 0.0
    E1 = E1_new

    # S: explicit diffusion + reaction (using current E1)
    lapS = laplacian_2d(S, mask, dx)
    v = (kcat * E1 * S) / (Km_mM + S + 1e-16)   # mM/s, using E1 (mM)
    v[~mask] = 0.0
    S_new = S + dt * (D_S * lapS - v)
    S_new[~mask] = 0.0
    S_new[S_new < 0] = 0.0
    S = S_new

    # I1: implicit diffusion + production v
    v_vec = v[mask_positions[:,0], mask_positions[:,1]]            # production on masked nodes
    I1_vec = I1[mask_positions[:,0], mask_positions[:,1]]
    rhs = I1_vec + dt * v_vec
    I1_new_vec = lu.solve(rhs)
    I1_new = np.zeros_like(I1)
    I1_new[mask_positions[:,0], mask_positions[:,1]] = I1_new_vec
    I1_new[~mask] = 0.0
    I1_new[I1_new < 0] = 0.0
    I1 = I1_new

    # record totals and optionally save frames
    if (n % frame_interval == 0) or (n == steps - 1):
        t = (n+1)*dt
        totalS = np.sum(S[mask]) * area_per_cell
        totalI = np.sum(I1[mask]) * area_per_cell
        times.append(t); totS.append(totalS); totI.append(totalI)

        # render side-by-side figure (S and I1)
        fig, axs = plt.subplots(1,2, figsize=(10,4))
        im0 = axs[0].imshow(np.where(mask, S, np.nan), origin='lower',
                            extent=[x.min(), x.max(), y.min(), y.max()],
                            interpolation='nearest')
        axs[0].add_artist(plt.Circle((0,0), R, fill=False, color='white', lw=1))
        axs[0].add_artist(plt.Circle((0,0), shell_inner, fill=False, color='yellow', lw=1, linestyle='--'))
        axs[0].set_title(f"S (mM) t={t:.3f}s\nTotal S={totalS:.6e}")
        axs[0].axis('off')
        plt.colorbar(im0, ax=axs[0], fraction=0.046, pad=0.02, label='S (mM)')

        im1 = axs[1].imshow(np.where(mask, I1, np.nan), origin='lower',
                            extent=[x.min(), x.max(), y.min(), y.max()],
                            interpolation='nearest')
        axs[1].add_artist(plt.Circle((0,0), R, fill=False, color='white', lw=1))
        axs[1].add_artist(plt.Circle((0,0), shell_inner, fill=False, color='yellow', lw=1, linestyle='--'))
        axs[1].set_title(f"I1 (mM)\nTotal I1={totalI:.6e}")
        axs[1].axis('off')
        plt.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.02, label='I1 (mM)')

        fig.tight_layout()
        frames_S.append(fig_to_rgb_array(fig))
        plt.close(fig)

    # small progress print (coarse)
    if (n+1) % max(1, steps//10) == 0:
        print(f"Step {n+1}/{steps} (t={(n+1)*dt:.3f}s)  totalS={totalS:.6e}  totalI={totalI:.6e}")

# ------------------ Save outputs ------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
gif_path = os.path.join(out_dir, f"S_I1_evolution_{timestamp}.gif")
imageio.mimsave(gif_path, frames_S, fps=fps)

csv_grid = os.path.join(out_dir, f"pde_results_grid_{timestamp}.csv")
ys, xs = np.nonzero(mask)
coords = np.column_stack((X[ys, xs].ravel(), Y[ys, xs].ravel(),
                          S[ys, xs].ravel(), I1[ys, xs].ravel(), E1[ys, xs].ravel()))
pd.DataFrame(coords, columns=["x_um","y_um","S_mM","I1_mM","E1_mM"]).to_csv(csv_grid, index=False)

csv_ts = os.path.join(out_dir, f"sim_timeseries_{timestamp}.csv")
pd.DataFrame({"t_s": times, "totalS_mM_um2": totS, "totalI_mM_um2": totI}).to_csv(csv_ts, index=False)

np.save(os.path.join(out_dir, "S_final.npy"), S)
np.save(os.path.join(out_dir, "I1_final.npy"), I1)
np.save(os.path.join(out_dir, "E1_field.npy"), E1)

# final diagnostics
initial_S_total = np.sum(S0_annulus_mM * annulus_mask) * area_per_cell  # sanity: estimated initial from geometry
final_S_total = totS[-1]
final_I_total = totI[-1]
consumed = initial_S_total - final_S_total
pct_consumed = (consumed / initial_S_total * 100.0) if initial_S_total > 0 else np.nan

print("\n--- Final diagnostics ---")
print(f"Initial total S (mM·µm^2) ~ {initial_S_total:.6e}")
print(f"Final total S   (mM·µm^2) = {final_S_total:.6e}")
print(f"Total I1 formed (mM·µm^2) = {final_I_total:.6e}")
print(f"Consumed = {consumed:.6e}  -> {pct_consumed:.2f}% of initial S")
print(f"Outputs (GIF + CSVs + npy) saved in: {out_dir}")
print(f"Combined GIF: {gif_path}")
print(f"Grid CSV: {csv_grid}")
print(f"Timeseries CSV: {csv_ts}")
""